# Install packages

In [1]:
!nvidia-smi

Fri Apr 21 21:39:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! pip install transformers
! pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 27.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# import all packages

In [3]:
from transformers import BertTokenizer, GPT2LMHeadModel, TextGenerationPipeline,GPT2Tokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

import re
import pandas as pd
import datetime
import torch
import os

## Define data cleaning function to remove symbols

In [4]:
def cleaning(s):
    s = str(s)
    s = re.sub('\s\W',' ',s)
    s = re.sub('\W,\s',' ',s)
    s = re.sub("\d+", "", s)
    s = re.sub('\s+',' ',s)
    s = re.sub('[!@#$_]', '', s)
    s = s.replace("co","")
    s = s.replace("https","")
    s = s.replace("[\w*"," ")
    return s

## Converting CVS file to TXT file

In [ ]:
def cvs2txt(origin_filename,saving_file):

    df = pd.read_csv(origin_filename, encoding="ISO-8859-1") 
    df = df.dropna()
    text_data = open(saving_file, 'w')
    for idx, item in df.iterrows():
        article = cleaning(item["Article"])
        text_data.write(article)
    text_data.close()
    print('txt data is saved!')

In [5]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size
    )
    return dataset

In [6]:
def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, 
        mlm=mlm,
    )
    return data_collator

## Define Training function

In [7]:
def train(train_file_path,model_name,output_dir,overwrite_output_dir,per_device_train_batch_size,num_train_epochs,save_steps,date):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print('>>>>>> Using %s to train your model <<<<<<'%device)
    tokenizer = BertTokenizer.from_pretrained(model_name)
    train_dataset = load_dataset(train_file_path, tokenizer)
    data_collator = load_data_collator(tokenizer)
    # tokenizer.save_pretrained(output_dir)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    # model.save_pretrained(output_dir)
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=overwrite_output_dir,
        per_device_train_batch_size=per_device_train_batch_size,
        num_train_epochs=num_train_epochs,
        optim='adamw_torch',
        )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        )
    model_path = output_dir+'_'+str(save_steps)+'_'+str(num_train_epochs)+'_'+str(date)
    trainer.train()
    trainer.save_model(model_path)
    print('>>>>> New model is saved! <<<<<')

In [ ]:
import os
origin_filename = "Articles.csv"
saving_file = "Articles.txt"
cvs2txt(origin_filename,saving_file)


train_file_path = "Articles.txt"


txt data is saved!


In [8]:
today = datetime.date.today()

In [9]:
file_path = 'result_content.txt'

In [10]:
import numpy as np

In [12]:
data = np.genfromtxt(file_path,delimiter='<END>',dtype=str)
print(data[0])

['范廷颂是什么时候被任为主教的？'
 "范廷颂是什么时候被任为主教的？答：1963年。['范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；1990年被擢升为天主教河内总教区宗座署理；1994年被擢升为总主教，同年年底被擢升为枢机；2009年2月离世。范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生；童年时接受良好教育后，被一位越南神父带到河内继续其学业。范廷颂于1940年在河内大修道院完成神学学业。范廷颂于1949年6月6日在河内的主教座堂晋铎；及后被派到圣女小德兰孤儿院服务。1950年代，范廷颂在河内堂区创建移民接待中心以收容到河内避战的难民。1954年，法越战争结束，越南民主共和国建都河内，当时很多天主教神职人员逃至越南的南方，但范廷颂仍然留在河内。翌年管理圣若望小修院；惟在1960年因捍卫修院的自由、自治及拒绝政府在修院设政治课的要求而被捕。1963年4月5日，教宗任命范廷颂为天主教北宁教区主教，同年8月15日就任；其牧铭为「我信天主的爱」。由于范廷颂被越南政府软禁差不多30年，因此他无法到所属堂区进行牧灵工作而专注研读等工作。范廷颂除了面对战争、贫困、被当局迫害天主教会等问题外，也秘密恢复修院、创建女修会团体等。1990年，教宗若望保禄二世在同年6月18日擢升范廷颂为天主教河内总教区宗座署理以填补该教区总主教的空缺。1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理；同年11月26日，若望保禄二世擢升范廷颂为枢机。范廷颂在1995年至2001年期间出任天主教越南主教团主席。2003年4月26日，教宗若望保禄二世任命天主教谅山教区兼天主教高平教区吴光杰主教为天主教河内总教区署理主教；及至2005年2月19日，范廷颂因获批辞去总主教职务而荣休；吴光杰同日真除天主教河内总教区总主教职务。范廷颂于2009年2月22日清晨在河内离世，享年89岁；其葬礼于同月26日上午在天主教河内总教区总主教座堂举行。']"]


In [13]:
qa_data = []

In [14]:
for i in range(len(data)):
  qa_data.append(data[i][1])

In [15]:
train_file_path = 'Chinese_QA_corpus.txt'

In [16]:
np.savetxt(train_file_path,qa_data,fmt='%s')

In [18]:
os.environ["WANDB_DISABLED"] = "true"
model_name = 'uer/gpt2-chinese-cluecorpussmall'
output_dir = 'gpt2-fine-tune/'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 10
save_steps = 1000

In [19]:
path = os.getcwd()
new_output_dir = output_dir+str(today)
path = os.path.join(path,new_output_dir)
# print(f'path:{path}')
isExist = os.path.exists(path)

In [20]:
if not isExist:
    os.makedirs(path)
    print('>>>>>> Model Folder is created <<<<<<')

>>>>>> Model Folder is created <<<<<<


# Model training

In [21]:
train(
  train_file_path=train_file_path,
  model_name=model_name,
  output_dir=path,
  overwrite_output_dir=overwrite_output_dir,
  per_device_train_batch_size=per_device_train_batch_size,
  num_train_epochs=num_train_epochs,
  save_steps=save_steps,
  date=today)

>>>>>> Using cuda to train your model <<<<<<


/usr/local/lib/python3.9/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
500,2.924700
1000,2.790000
1500,2.708800
2000,2.634800
2500,2.590100
3000,2.522500
3500,2.475000
4000,2.428100
4500,2.377500
5000,2.349000


>>>>> New model is saved! <<<<<


# Model Evaluation

### Before training

### 英文

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)


text_generator = TextGenerationPipeline(model, tokenizer)   
output = text_generator("The Sindh government", max_length=100, do_sample=True)
print(f'output:{output}')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


output:[{'generated_text': 'The Sindh government on Thursday said it will stop mining in its territory, even where the mining permits belong without the approval of Congress. The Sindh government on Thursday said it will stop mining IOUs in its territory, even where the mining permits belong without the approval of Congress.\n\nIn 2012, the Supreme Court cleared an exemption for the mining in Haryana and Rajasthan. Since then, the Haryana government has said that "it has not taken any action on the'}]


### 中文

In [24]:
from transformers import BertTokenizer, GPT2LMHeadModel, TextGenerationPipeline
tokenizer = BertTokenizer.from_pretrained("uer/gpt2-chinese-cluecorpussmall")
model = GPT2LMHeadModel.from_pretrained("uer/gpt2-chinese-cluecorpussmall")
text_generator = TextGenerationPipeline(model, tokenizer)   
text_generator("江苏路街道在上海市的什么地方？", max_length=100, do_sample=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': '江苏路街道在上海市的什么地方？ 为 什 么 说 无 论 是 江 苏 路 街 道 ， 还 是 江 苏 路 街 道 都 有 着 悠 长 的 历 史 ， 无 论 是 江 苏 路 街 道 还 是 江 苏 路 街 道 。 无 论 是 苏 州 河 街 道 ， 还 是 江 苏 路 街 道 ， 无 论 在 南 京 路 到 江 苏 路 这 一 段 ， 还 是 江 苏 路 街 道 ， 无'}]

### After training

### 英文

In [ ]:
checkpoint = 'checkpoint-5000' ### choose the saved checkpoint
saved_model = output_dir + checkpoint

tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(saved_model)


text_generator = TextGenerationPipeline(model, tokenizer)   
output = text_generator("The Sindh government", max_length=100, do_sample=True)
print(f'output:{output}')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'GPT2Tokenizer'.


TypeError: ignored

### 中文

In [44]:
checkpoint = '2023-04-21/checkpoint-' ### choose the saved checkpoint
# saved_model = output_dir + checkpoint
for i in range(500,50000,500):
  saved_model = output_dir + checkpoint + str(i)
  tokenizer = BertTokenizer.from_pretrained(model_name)
  model = GPT2LMHeadModel.from_pretrained(saved_model)


  text_generator = TextGenerationPipeline(model, tokenizer)
  output = text_generator("江苏路街道在上海市的什么地方？", max_length=100, do_sample=True)
  print('Checkpoint:',i,'output:',output)



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Checkpoint: 500 output: [{'generated_text': "江苏路街道在上海市的什么地方？ 答 ： 上 海 市 浦 东 新 区 的 上 元 路 街 道 ， 上 海 市 浦 东 新 区 天 桥 区 的 下 元 路 街 道 。 ['上 海 市 金 山 区 浦 阳 镇 ， 是 上 海 市 上 海 市 浦 东 新 区 下 辖 的 一 个 镇 ， 位 于 浦 东 新 区 的 上 海 市 浦 东 新 区 浦 阳 镇 。 上 海 市 金"}]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Checkpoint: 1000 output: [{'generated_text': "江苏路街道在上海市的什么地方？ 答 ： 浙 江 省 宁 波 市 。 ['江 苏 路 ， 位 于 浙 江 省 宁 波 市 的 哪 个 地 方 ？ 答 ： 浙 江 省 宁 波 市 的 其 中 一 个 街 道 。 ['江 苏 路 ， 是 古 代 的 一 条 中 心 道 路 ， 为 宁 波 江 南 带 的 主 要 道 路 之 一 ， 也 是 中 国 近 代 思 想 史"}]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Checkpoint: 48500 output: [{'generated_text': "江苏路街道在上海市的什么地方？ 答 ： 苏 州 面 积 最 大 的 法 寄 食 品 集 中 区 。 ['苏 州 面 包 店 是 中 国 苏 州 面 包 店 的 一 种 ， 在 小 城 市 还 有 很 多 名 店 面 积 达 500 平 方 米 ， 公 司 设 在 苏 州 吴 中 区 上 海 市 浦 东 新 区 陆 家 嘴 ， 市 南 区 是 苏 州 高 新 区 的 高"}]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Checkpoint: 49000 output: [{'generated_text': "江苏路街道在上海市的什么地方？ 答 ： 中 山 公 园 的 南 侧 。 ['上 海 市 静 安 区 是 江 苏 省 静 安 区 下 辖 的 一 个 区 域 。 是 江 苏 省 的 第 二 大 商 业 区 ， 也 是 江 苏 省 最 大 的 服 装 城 ， 全 国 从 严 格 意 义 上 说 是 高 档 服 装 城 市 。 1992 年 被 中 华 人 民 共 和 国 国"}]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Checkpoint: 49500 output: [{'generated_text': "江苏路街道在上海市的什么地方？ 答 ： 人 民 路 、 延 安 中 路 、 武 夷 路 和 浦 东 新 区 陆 家 嘴 ， 以 及 上 海 东 昌 南 路 等 。 ['徐 汇 区 委 为 江 苏 省 国 家 级 经 济 技 术 开 发 区 ， 由 徐 汇 区 管 理 委 员 会 管 辖 ， 位 于 徐 汇 区 北 部 ， 东 以 陕 西 南 路 与 黄 浦 区 瑞"}]
